# Exercise 9: Effect size and power analysis

## Table of Contents

* Effect sizes
* Power analysis
* Simulating effect of power on reported effect size

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.stats as sps
import os

# For retina displays only 
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
%matplotlib inline

## Effect sizes

In [ ]:
# The fuctions below calculates the cohen d and t statistic for two independant d1 and d2 
def cohend(d1, d2):
	# calculate the size of samples
	n1, n2 = len(d1), len(d2)
	# calculate the variance of the samples
	v1, v2 = np.var(d1, ddof=1), np.var(d2, ddof=1)
	# calculate the pooled standard deviation
	s = np.sqrt(((n1 - 1) * v1 + (n2 - 1) * v2) / (n1 + n2 - 2))
	# calculate the means of the samples
	u1, u2 = np.mean(d1), np.mean(d2)
	# calculate the effect size
	return (u2 - u1) / s

def t_stat_ind(d1, d2):
	# calculate the size of samples
	n1, n2 = len(d1), len(d2)
    # calculate the variance of the samples
	v1, v2 = np.var(d1, ddof=1), np.var(d2, ddof=1)
	# calculate the pooled standard deviation
	s = np.sqrt(v1 / n1 + v2 / n2)
	# calculate the means of the samples
	u1, u2 = np.mean(d1), np.mean(d2)
	# calculate the effect size
	return (u2 - u1) / s 

In [ ]:
# Excercise: Finish this function for GlassD
def GlassD(d_control, d_experiment):
	# calculate the appropriate unceratinty 
    s = ???
	# calculate the means of the samples
	u_control, u_experiment = np.mean(d_control), np.mean(d_experiment)
	# calculate the effect size
	return ???

In [ ]:
# Exercise: Write a function for the one-sample t-statsic 
def t_stat_1samp(d):
    ???

In [ ]:
# Exercise: Generate two independent data sets and compare effect sizes given by cohend, t_stat_ind, and GlassD








## Power analysis

Here we're going to use `statsmodels.stats.power` to do some power analyses. This package has methdods for many different types of parametric tests, but we're going to focus on the two-sample t-test, `TTestIndPower`.

In [ ]:
import statsmodels.stats.power as smp

In [ ]:
# parameters for power analysis
effect = 0.8
alpha = 0.05
power = 0.8

# perform power analysis
analysis = smp.TTestIndPower()
result = analysis.solve_power(effect, power=power, nobs1=None, ratio=1.0, alpha=alpha)
print(f'Sample Size: {result:0.5}')

In [ ]:
# Excersise: create a plot of sample size vs power for various power sizes. What happens if effect size drops? 

effect = 0.8 # Expressed in cohen's d
alpha = 0.05
powers = np.linsapce(0.01, .99, 100)
sample_sizes = []
for power in powers:
    ???

fig, ax = plt.subplots()
???

In [ ]:
# Excerise: use .power to figure out the power of comparing the following data sets
X1 = np.random.normal(loc=0, scale=2, size=10)
X2 = np.random.normal(loc=0.5, scale=2, size=10)






In [ ]:
# Exercise: use .plot_power to plot power with number of observations or effect size on x-axis






## Simulating effect of sample size on reported effect size

In [ ]:
sample_sizes = np.arange(5, 100, 1)  # different sample sizes to simulate
alpha = 0.05;                       # our alpha threshold
N_rep = 2000;                        # number of simulations to perform

m1 = 0
m2 = 1
s = 2

powers = np.empty(len(sample_sizes))
effect_sizes = np.empty((len(sample_sizes), N_rep))
sig_mask = np.empty((len(sample_sizes), N_rep), dtype='Bool') 

for i, size in enumerate(sample_sizes):
    # calculating power for each sample size
    analysis = smp.TTestIndPower()
    power = analysis.power((m2 - m1)/s, size, alpha)
    powers[i] = power
    for j in range(N_rep):
        # draw two samples from each distribution
        sample1 = np.random.normal(m1, s, size)
        sample2 = np.random.normal(m2, s, size)
        
        # record acutal effect size for every simulation
        sim_effect_size = cohend(sample1, sample2) 
        effect_sizes[i, j] = sim_effect_size
        
        # assess whether there is a statistically significant difference in means
        p_value = sps.ttest_ind(sample1, sample2)[1]
        
        # if significant, record some results
        if p_value < alpha:
            sig_mask[i, j] = True
        else:
            sig_mask[i, j] = False



In [ ]:
# Calculating statistics 
all_10 = np.percentile(effect_sizes, 10, axis=1)
all_50 = np.percentile(effect_sizes, 50, axis=1)
all_90 = np.percentile(effect_sizes, 90, axis=1) 

sig_10 = [ np.percentile(vals[mask], 10) for vals, mask in zip(effect_sizes, sig_mask)]
sig_50 = [ np.percentile(vals[mask], 50) for vals, mask in zip(effect_sizes, sig_mask)]
sig_90 = [ np.percentile(vals[mask], 90) for vals, mask in zip(effect_sizes, sig_mask)]

In [ ]:
# Plotting effect sizes as a function of sample sze
fig, ax = plt.subplots()
ax.plot(sample_sizes, sig_50, color='red', label='only sig.')
ax.fill_between(sample_sizes, sig_10, sig_90, alpha=.2, color='red')
ax.plot(sample_sizes, all_50, color='green', label='all')
ax.fill_between(sample_sizes, all_10, all_90, alpha=.2, color='green')
ax.axhline((m2 - m1)/s, color='.5', linewidth=1, linestyle='dotted')

ax.legend()
ax.set_xlabel('sample size')
ax.set_ylabel("effect size (Cohen's d)")

In [ ]:
# Plotting effect sizes as a function of power
fig, ax = plt.subplots()
ax.plot(powers, sig_50, color='red', label='only sig.')
ax.fill_between(powers, sig_10, sig_90, alpha=.2, color='red')
ax.plot(powers, all_50, color='green', label='all')
ax.fill_between(powers, all_10, all_90, alpha=.2, color='green')
ax.axhline((m2 - m1)/s, color='.5', linewidth=1, linestyle='dotted')

ax.legend()
ax.set_xlabel('power ($1 - \beta$)')
ax.set_ylabel("effect size (Cohen's d)")